In [1]:
from cProfile import label
from email import header
from ntpath import join
from os import sep
from statistics import mean, median
from tkinter.ttk import Separator
from xml.etree.ElementPath import xpath_tokenizer
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector
import numpy as np
import seaborn as sns
from scipy.signal import find_peaks
import pingouin as pg
from pingouin import homoscedasticity

C:\Users\kuba8\miniconda3\envs\julab\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#data, we would like to evaluate
healthy=['ZM001_b_data','ZM002_b_data','ZM004_b_data']
paralyzed=['SM001_bal_data','SM002_bal_data','SM003_bal_data','SM005_bal_data']
paralyzed=['SF006_bar_data','SF007_bar_data','SF010_bar_data','SF011_bar_data']
paralyzed=['SM021_bar_data','SM022_bar_data','SM023_bar_data']
names=paralyzed

In [3]:
#function which select peaks in signal and set borders for next calculation
def select2(x,rel_hig, wid,dis,a,b,promin=0):

    #initial gues of promminence
    if promin==0:
        promin=(max(x[a:b])-mean(x[a:b]))*0.5

    #function for searching peaks
    peaks, properties = find_peaks(-x[a:b], prominence=promin, rel_height=rel_hig, width=wid, distance=dis)

    #repeat, if number of peaks is bigger than 6
    if len(peaks)>6:
        promin=promin+0.002/20
        peaks,properties=select2(x, rel_hig, wid,dis,a,b,promin)
    properties["prominences"], properties["widths"]

    #returns borders of peaks(properties) and order index of peaks
    return peaks, properties

In [4]:
#function for testing equality of means
def process(data,num,num2):

    #select specific points in signal
    peak1,prop1=select2(data[num],0.8,1,5.5,0,len(data))

    #widen area of peaks
    p=2
    x=np.array([])

    #make signal composed from exercises
    for j in range(0,len(peak1)):
        xpo=data[num2][round(prop1['left_ips'][j])-p:round(prop1['right_ips'][j])+p]
        x=np.append(x,xpo)
    return x

In [5]:
index=np.array([])
p_val=pd.DataFrame([])
big_table=pd.DataFrame([])
for l in range(0,4):
    dataset=pd.DataFrame([])   
    for k in range(0,len(names)):

        #load data
        data=pd.read_csv(join(r'C:\Users\kuba8\MATLAB Drive\projekt oblicej\dataPython',names[k]+'_new.txt'),sep=' ',header=None)
        data=data.transpose()
        #data=normalize(data)

        #get specific area of peaks
        data=process(data,1,l)
        df = pd.DataFrame({'value':data, 'label': names[k]})#random.choices(data,k=20)})
        dataset=pd.concat([dataset,df], axis=0)
        
        #making big table
        table_h = pd.DataFrame({'sex':[names[k][1]],'status':[names[k][0]],'ID_number':[names[k][2:5]],'point':[names[k][1]]})
        big_table=pd.concat([big_table,table_h], axis=0)
    
    if l<12:#12
        kr=pg.kruskal(dataset,dv='value',between='label')
        if kr['p-unc'][0]<0.05:
            hod=False
        else:
            hod=True
        kr=pd.DataFrame({'W':kr['H'],'pval':kr['p-unc'],'equal':hod})
    else:
        kr=homoscedasticity(dataset,dv='value',group='label')
        kr=pd.DataFrame({'W':kr['W'],'pval':kr['pval'],'equal':kr['equal_var']})
    #kr=kruskal(dataset.groupby(by='label'))
    p_val=pd.concat([p_val,kr])
    index=np.append(index,l)
p_val['point']=index
    #if kr['pval'][0]>0.05:
    #    i+=1
    #p_val=np.append(p_val,kr[1])

In [6]:
p_val

,W,pval,equal,point
Kruskal,56.436787,5.557858e-13,False,0.0
Kruskal,179.375458,1.119732e-39,False,1.0
Kruskal,43.166799,4.231055e-10,False,2.0
Kruskal,78.244009,1.022184e-17,False,3.0


In [7]:
points=p_val[p_val["equal"] == True]

In [8]:
points

,W,pval,equal,point
